In [ ]:
# default_exp core

# MRtrix3notebook

> Develop MRtrix3-based pipelines using Jupyter notebooks

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import shutil
import logging
import subprocess
import shlex
from pathlib import Path
from collections import deque
import pprint

logging.basicConfig()

In [ ]:
# export
class Comp:
    def __init__(self, dry_run=False, env=os.environ.copy(), loglevel='INFO'):
        self.dry_run = dry_run
        
        self.jobs = deque()
        
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(loglevel)
        
        # use directory of python binary
        python = shutil.which('python') # None if executable not found
        if python is None:
            raise ImportError("could not find python in Path")
        
        if 'PATH' in env:
            env['PATH'] = os.pathsep.join([str(Path(python).parent), env.get('PATH')])
        self.logger.debug('env: '+pprint.pformat(env))
        self.env = env
        
    
    def _execute(self, cmd):
        if not self.dry_run:
            self.logger.debug('running: ' + cmd)
            subprocess.check_call(cmd, shell=True, env=self.env)
        else:
            self.logger.info('dry_run: ' + cmd)
        return 
        
    def run(self):
        while self.jobs:
            self._execute(self.jobs.popleft())
    
    def __add__(self, x):
        if isinstance(x, list):
            x = shlex.join(x)
        elif not isinstance(x, str):
            raise TypeError(str(type(x)))
        self.logger.debug('add: '+x)
        self.jobs.append(x)
        return self
            
    def __iadd__(self, x):
        return self.__add__(x)
            
    def __repr__(self):
        return 'computer %i jobs: %s' % (self.__len__(), str(self.jobs))
    
    def __str__(self):
        return self.__repr__()
    
#     def __next__(self):
#         return self.jobs.get()
    
    def __len__(self):
        return len(self.jobs)
    
    def __enter__(self):
        self.logger.debug('Entering Comp')
        return self

    def __exit__(self, exc_type, exc, exc_tb):
        self.logger.debug('Exiting Comp')
        self.run()

In [ ]:
with Comp(dry_run=True) as run:
    run.logger.info('start')
    run += 'mrinfo --version'
    run += ['dcminfo', '--version']
    run.logger.info('done')

INFO:__main__:start
INFO:__main__:done
INFO:__main__:dry_run: mrinfo --version
INFO:__main__:dry_run: dcminfo --version


In [ ]:
c = Comp(loglevel='INFO')
c += 'dcminfo --version'

c.run()

In [ ]:
c = Comp(loglevel='INFO')
c += ['mrinfo', '-version']
print(c)
c.run()
print(c)

computer 1 jobs: deque(['mrinfo -version'])
computer 0 jobs: deque([])
